<a href="https://colab.research.google.com/github/v2sh1t/Colab-Stable-Diffusion/blob/main/JefSDForge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jef's SD Forge Colab
## Fork from https://github.com/RedDeltas/SDForge-Colab

The notebook is intended to make it easy to use [Stable Diffusion WebUI Forge](https://github.com/lllyasviel/stable-diffusion-webui-forge) in Google Colab. If you run into any problems or have any suggestions, please create an Issue on Github.

## Parameters


*   **USE_GOOGLE_DRIVE** - Stores the Forge installation in your GDrive, this makes it easy to store your generated images, checkpoints, installed extensions etc between sessions
*   **UPDATE_FORGE** - Update your Forge installation when you run this notebook
*   **INSTALL_DEPS** - Installs optional dependencies such as `insightface`
*   **ALLOW_EXTENSION_INSTALLATION** - Allow installing extensions through the UI **Warning**: It is recommended to have **USE_USERNAME_AND_PASSWORD** turned on if you're using this, otherwise anyone with your Gradio URL could install malicious extensions on your instance
*   **USE_USERNAME_AND_PASSWORD** - Require inputting a username and password when accessing the Gradio URL
*   **USERNAME** - Username to use when accessing the Gradio URL
*   **PASSWORD** - Password to use when accessing the Gradio URL



In [6]:
from google.colab import drive

USE_GOOGLE_DRIVE = False  #@param {type:"boolean"}
UPDATE_FORGE = True  #@param {type:"boolean"}
INSTALL_DEPS = True #@param {type:"boolean"}
ALLOW_EXTENSION_INSTALLATION = True #@param {type:"boolean"}
USE_USERNAME_AND_PASSWORD = True #@param {type:"boolean"}
USERNAME = "" # @param {type:"string"}
PASSWORD = "" # @param {type:"string"}

WORKSPACE = '/content/stable-diffusion-webui-forge'

if USE_GOOGLE_DRIVE:
  print("📂 Connecting to Google Drive...")
  drive.mount('/content/drive')

  WORKSPACE = "/content/drive/MyDrive/stable-diffusion-webui-forge"
  %cd /content/drive/MyDrive

![ ! -d $WORKSPACE ] && echo -= Initial setup SDForge =- && git clone --config core.filemode=false https://github.com/lllyasviel/stable-diffusion-webui-forge.git

In [ ]:
#@title Download Models
!pip3 install "huggingface_hub[hf_transfer]"

%cd $WORKSPACE
import os, subprocess, sys

repo_id = "" #@param {'type': 'string'}
allow_patterns = "" #@param {'type': 'string'}
dir = "" #@param {'type': 'string'}

def install_huggingface_hub():
  subprocess.check_call([sys.executable, "-m", "pip", "install", "huggingface_hub[hf_transfer]"])

def download_models():
  try:
    from huggingface_hub import snapshot_download
    os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
    snapshot_download(
      repo_id=f"{repo_id}",
      allow_patterns=f"{allow_patterns}",
      local_dir=f"{WORKSPACE}/models/{dir}"
    )
  except ImportError:
    print("huggingface_hub is not installed. Installing now...")
    install_huggingface_hub()
    from huggingface_hub import snapshot_download

if __name__ == "__main__":
  download_models()

In [ ]:
#@title Run Forge on Web

%cd $WORKSPACE

extra_args = []

if UPDATE_FORGE:
  !echo -= Updating SDForge =-
  !git pull

if INSTALL_DEPS:
  !echo -= Install dependencies =-
  !pip3 install insightface

if ALLOW_EXTENSION_INSTALLATION:
  extra_args.append("--enable-insecure-extension-access")

if USE_USERNAME_AND_PASSWORD:
  extra_args.append(f"--gradio-auth {USERNAME}:{PASSWORD}")

extra_args_concat = " ".join(extra_args)

!python launch.py --share {extra_args_concat}